# Validation of the DP and EMT 3th, 4th and 6th Order VBR SynGen against SP domain

## Run C++ examples

In [ ]:
import os
import subprocess

#%matplotlib widget

name = ['SP_ReducedOrderSG_VBR_Load_Fault', 'DP_ReducedOrderSG_VBR_Load_Fault', 'EMT_ReducedOrderSG_VBR_Load_Fault']
options = ['SGModel=3', 'SGModel=4', 'SGModel=6a', 'SGModel=6b']

dpsim_path = subprocess.Popen(['git', 'rev-parse', '--show-toplevel'], stdout=subprocess.PIPE).communicate()[0].rstrip().decode('utf-8')

path_exec = dpsim_path + '/build/dpsim/examples/cxx/'
for i in range(len(name)):
    for j in range (len(options)):
        sim = subprocess.Popen([path_exec + name[i], '-o', options[j]], stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
        print(sim.communicate()[0].decode())
        print(path_exec + name[i])

### Load Results

In [ ]:
from villas.dataprocessing.readtools import *
from villas.dataprocessing.timeseries import *
from villas.dataprocessing.timeseries import TimeSeries as ts
import matplotlib.pyplot as plt
import re
import numpy as np
import math
import os

work_dir = os.getcwd() + "/logs/"
file_name_3Order = 'SynGen3Order_VBR_Load_Fault'
file_name_4Order = 'SynGen4Order_VBR_Load_Fault'
file_name_6aOrder = 'SynGen6aOrder_VBR_Load_Fault'
file_name_6bOrder = 'SynGen6bOrder_VBR_Load_Fault'

ts_dpsim_SP = []
ts_dpsim_SP_shift = []
ts_dpsim_DP = []
ts_dpsim_DP_shift = []
ts_dpsim_EMT = []
file_names = [file_name_3Order, file_name_4Order, file_name_6aOrder, file_name_6bOrder]
for i in range(len(file_names)):
    path_logfile = work_dir + "SP_" + file_names[i] + "/" + "SP_" + file_names[i] + ".csv"
    ts_dpsim_SP.append(read_timeseries_dpsim(path_logfile))
    ts_dpsim_SP_shift.append(ts.frequency_shift_list(ts_dpsim_SP[i], 60))
    
    path_logfile = work_dir + "DP_" + file_names[i] + "/" + "DP_" + file_names[i] + ".csv"
    ts_dpsim_DP.append(read_timeseries_dpsim(path_logfile))
    ts_dpsim_DP_shift.append(ts.frequency_shift_list(ts_dpsim_DP[i], 60))
    print(path_logfile)
    
    path_logfile = work_dir + "EMT_" + file_names[i] + "/" + "EMT_" + file_names[i] + ".csv"
    ts_dpsim_EMT.append(read_timeseries_dpsim(path_logfile))

### Declare plot function

In [ ]:
width = 16
height = 4
color_EMT = 'black'
color_SP = 'C10'
color_DP = 'C11'
linestyle_EMT = '-'
linestyle_SP = '-'
linestyle_DP = 'dashed'
common_time=50e-6
roi_start=0
roi_end=4.5
roi = range(int(roi_start/common_time), int(roi_end/common_time))

def plot_etorque(SGOrder):
    if SGOrder==3:
        idx=0
    if SGOrder==4:
        idx=1
    if SGOrder==6:
        # Marconato's model
        idx=2
    if SGOrder==7:
        # Anderson's model
        idx=3
        
    time = ts_dpsim_EMT[idx]["Te"].interpolate(common_time).time[roi]
    values_emt = ts_dpsim_EMT[idx]["Te"].interpolate(common_time).values[roi]
    values_sp = ts_dpsim_SP[idx]["Te"].interpolate(common_time).values[roi]
    values_dp = ts_dpsim_DP[idx]["Te"].interpolate(common_time).values[roi]
        
    fig = plt.figure(figsize=(width, height))
    plt.plot(time, values_emt, label='EMT', linestyle=linestyle_EMT, color=color_EMT, linewidth=3)    
    plt.plot(time, values_sp, label='SP',  color=color_SP, linestyle=linestyle_SP,  linewidth=3) 
    plt.plot(time, values_dp, label='DP',  color=color_DP, linestyle=linestyle_DP, linewidth=3)     
    plt.ylabel('Electrical torque (p.u.)')
    plt.xlabel('Time (s)')
    plt.xlim(0, 4.5)
    plt.grid()
    plt.legend(loc='lower right')
    plt.show()
    
    #calculate RMSE
    nom_value = 0.540541 # power flow result
    rmse_emt = ts_dpsim_SP[idx]["Te"].rmse(ts_dpsim_SP[idx]["Te"].interpolate(common_time), ts_dpsim_EMT[idx]["Te"].interpolate(common_time))
    print('RMSE EMT = {:.6f} (pu), which is {:.3f}% of the nominal value = {:.3f} (pu) '.format(rmse_emt, rmse_emt/nom_value*100, nom_value))
    rmse_dp = ts_dpsim_SP[idx]["Te"].rmse(ts_dpsim_SP[idx]["Te"].interpolate(common_time), ts_dpsim_DP[idx]["Te"].interpolate(common_time))
    print('RMSE DP = {:.6f} (pu), which is {:.3f}% of the nominal value = {:.3f} (pu) '.format(rmse_dp, rmse_dp/nom_value*100, nom_value))
    
    return (rmse_emt/nom_value*100, rmse_dp/nom_value*100)
    
    
def plot_elecVar(SGOrder, variable_EMT, variable_SP, variable_DP, ylabel):
    if SGOrder==3:
        idx=0
    if SGOrder==4:
        idx=1
    if SGOrder==6:
        # Marconato's model
        idx=2
    if SGOrder==7:
        # Anderson's model
        idx=3
    
    time = ts_dpsim_EMT[idx][variable_EMT].interpolate(common_time).time[roi]
    values_emt = ts_dpsim_EMT[idx][variable_EMT].interpolate(common_time).values[roi]
    values_sp = np.sqrt(2/3) * ts_dpsim_SP_shift[idx][variable_SP].interpolate(common_time).values[roi]
    values_dp = np.sqrt(2/3) * ts_dpsim_DP_shift[idx][variable_DP].interpolate(common_time).values[roi] 
        
    fig = plt.figure(figsize=(width, height))
    plt.plot(time, values_emt, label='EMT', linestyle=linestyle_EMT, color=color_EMT, linewidth=3)    
    plt.plot(time, values_sp, label='SP',  color=color_SP, linestyle=linestyle_SP,  linewidth=3) 
    plt.plot(time, values_dp, label='DP',  color=color_DP, linestyle=linestyle_DP, linewidth=3) 
    plt.ylabel(ylabel)
    plt.xlabel('Time (s)')
    plt.xlim(0.9, 1.2)
    plt.grid()
    plt.legend(loc='lower right')
    plt.show()
    
    #calculate RMSE
    ts_dpsim_SP_real = TimeSeries(variable_SP, ts_dpsim_SP_shift[idx][variable_SP].time, np.sqrt(2/3) * ts_dpsim_SP_shift[idx][variable_SP].values.real)
    rmse_emt = ts_dpsim_SP_real.rmse(ts_dpsim_SP_real.interpolate(common_time), ts_dpsim_EMT[idx][variable_EMT].interpolate(common_time))
    nom_value = abs(max(ts_dpsim_SP_shift[idx][variable_SP].values[0:int(0.5/common_time)]))
    print('RMSE EMT = {:.6f} (pu), which is {:.3f}% of the nominal value = {:.3f} (pu) '.format(rmse_emt, rmse_emt/nom_value*100, nom_value))
    
    ts_dpsim_DP_real = TimeSeries(variable_DP, ts_dpsim_DP_shift[idx][variable_DP].time, np.sqrt(2/3) * ts_dpsim_DP_shift[idx][variable_DP].values.real)
    rmse_dp = ts_dpsim_SP_real.rmse(ts_dpsim_SP_real.interpolate(common_time), ts_dpsim_DP_real.interpolate(common_time))
    print('RMSE DP = {:.6f} (pu), which is {:.3f}% of the nominal value = {:.3f} (pu) '.format(rmse_dp, rmse_dp/nom_value*100, nom_value))
    
    return (rmse_emt/nom_value*100, rmse_dp/nom_value*100)

### Electrical torque

#### 3th order SG

In [ ]:
rmse_emt_3order, rmse_dp_3order = plot_etorque(3)

#### 4th order SG

In [ ]:
rmse_emt_4order, rmse_dp_4order = plot_etorque(4)

#### 6th order SG (Marconato's model)

In [ ]:
rmse_emt_6aorder, rmse_dp_6aorder = plot_etorque(6)

#### 6th order SG (Anderson-Fouad's model)

In [ ]:
rmse_emt_6border, rmse_dp_6border = plot_etorque(7)

### Assert

In [ ]:
tol = 0.001 # %

assert(rmse_emt_3order<tol)
assert(rmse_emt_4order<tol)
assert(rmse_emt_6aorder<tol)
assert(rmse_emt_6border<tol)
assert(rmse_dp_3order<tol)
assert(rmse_dp_4order<tol)
assert(rmse_dp_6aorder<tol)
assert(rmse_emt_6border<tol)

### Generator current

#### 3th order SG

In [ ]:
rmse_emt_3order, rmse_dp_3order = plot_elecVar(3, 'i_gen_0', 'i_gen_shift', 'i_gen_shift', 'Current (A)')

#### 4th order SG

In [ ]:
rmse_emt_4order, rmse_dp_4order = plot_elecVar(4, 'i_gen_0', 'i_gen_shift', 'i_gen_shift', 'Current (A)')

#### 6th order SG (Marconato's model)

In [ ]:
rmse_emt_6aorder, rmse_dp_6aorder = plot_elecVar(6, 'i_gen_0', 'i_gen_shift', 'i_gen_shift', 'Current (A)')

#### 6th order SG (Andeson-Fouad's model)

In [ ]:
rmse_emt_6border, rmse_dp_6aorder = plot_elecVar(7, 'i_gen_0', 'i_gen_shift', 'i_gen_shift', 'Current (A)')

### Assert

In [ ]:
tol = 0.001 # %

assert(rmse_emt_3order<tol)
assert(rmse_emt_4order<tol)
assert(rmse_emt_6aorder<tol)
assert(rmse_emt_6border<tol)
assert(rmse_dp_3order<tol)
assert(rmse_dp_4order<tol)
assert(rmse_dp_6aorder<tol)
assert(rmse_dp_6border<tol)

### Generator voltage

#### 3th order SG

In [ ]:
rmse_emt_3order, rmse_dp_3order = plot_elecVar(3, 'v_gen_0', 'v_gen_shift', 'v_gen_shift', 'Voltage (V)')

#### 4th order SG

In [ ]:
rmse_emt_4order, rmse_dp_4order = plot_elecVar(4, 'v_gen_0', 'v_gen_shift', 'v_gen_shift', 'Voltage (V)')

#### 6th order SG (Marconato's model)

In [ ]:
rmse_emt_6aorder, rmse_dp_6aorder = plot_elecVar(6, 'v_gen_0', 'v_gen_shift', 'v_gen_shift', 'Voltage (V)')

#### 6th order SG (Anderson-Fouad's model)

In [ ]:
rmse_emt_6border, rmse_dp_6border = plot_elecVar(7, 'v_gen_0', 'v_gen_shift', 'v_gen_shift', 'Voltage (V)')

### Assert

In [ ]:
tol = 0.001 # %

assert(rmse_emt_3order<tol)
assert(rmse_emt_4order<tol)
assert(rmse_emt_6aorder<tol)
assert(rmse_emt_6border<tol)
assert(rmse_dp_3order<tol)
assert(rmse_dp_4order<tol)
assert(rmse_dp_6aorder<tol)
assert(rmse_dp_6border<tol)